# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [1]:
import numpy as np
import openmc
import openmc.deplete as od

In [2]:
uox_model = openmc.Model()
mox_model = openmc.Model()

In [3]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 10000

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 10000

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
mox_model.Materials = openmc.Materials([mox, zirconium, water])

In [5]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')

water_region = +left & -right & +bottom & -top & +clad_outer_radius

moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

box = openmc.rectangular_prism(width=pitch, height=pitch,
                               boundary_type='reflective')

water_region = box & +clad_outer_radius

root_universe = openmc.Universe(cells=(fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(root_universe)
mox_model.geometry = openmc.Geometry(root_universe)

In [6]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.Source(space=point)

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/source.py:388: FutureWarning: This class is deprecated in favor of 'IndependentSource'
  warnings.warn("This class is deprecated in favor of 'IndependentSource'", FutureWarning)


In [7]:
# Settings
settings = openmc.Settings()
settings.source = source
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings

In [8]:
# Get cross section data
uox_cross_sections = od.MicroXS.from_model(uox_model, 
                                       reaction_domain = root_universe, 
                                       chain_file="/home/abachmann@anl.gov/openmc/chain_endfb71_pwr.xml")
mox_cross_sections = od.MicroXS.from_model(mox_model, 
                                       reaction_domain = root_universe, 
                                       chain_file="/home/abachmann@anl.gov/openmc/chain_endfb71_pwr.xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [20]:
# save cross sections from UOX model for use in Deplete Reactor
uox_cross_sections.to_csv("micro_xs.csv")

In [9]:
uox_ind_op = od.IndependentOperator(uox_model.Materials, 
                                uox_cross_sections, 
                                "/home/abachmann@anl.gov/openmc/chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(mox_model.Materials, 
                                mox_cross_sections, 
                                "/home/abachmann@anl.gov/openmc/chain_endfb71_pwr.xml")

In [10]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    np.ones(12)*30,
                                   power = 100*1e6,
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=31104000.0 (final operator evaluation)


In [11]:
! mv depletion_results.h5 uox_depletion_results.h5

In [12]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12)*30,
                                   power = 100*1e6,
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=100000000.0
[openmc.deplete] t=31104000.0 (final operator evaluation)


In [13]:
! mv depletion_results.h5 mox_depletion_results.h5

In [14]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [15]:
spent_uox = uox_results[-1].get_material('1')
spent_mox = mox_results[-1].get_material('5')

In [17]:
#printing in format for Cyclus recipe
for nuclide in spent_uox.nuclides:
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{zaid}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")

<nuclide>  <id>10020000.0</id> <comp>1.1822861416849385e-06</comp>  </nuclide>
<nuclide>  <id>20040000.0</id> <comp>0.0004390174203752439</comp>  </nuclide>
<nuclide>  <id>60120000.0</id> <comp>1.3401679582425634e-11</comp>  </nuclide>
<nuclide>  <id>60130000.0</id> <comp>0.00043887600532518397</comp>  </nuclide>
<nuclide>  <id>70150000.0</id> <comp>2.4660055129028634e-10</comp>  </nuclide>
<nuclide>  <id>80170000.0</id> <comp>7.290180639807623e-06</comp>  </nuclide>
<nuclide>  <id>290650000.0</id> <comp>3.4498157766469734e-16</comp>  </nuclide>
<nuclide>  <id>300660000.0</id> <comp>5.959272905013617e-14</comp>  </nuclide>
<nuclide>  <id>300670000.0</id> <comp>3.447260796424438e-13</comp>  </nuclide>
<nuclide>  <id>300680000.0</id> <comp>4.597619795129254e-13</comp>  </nuclide>
<nuclide>  <id>300700000.0</id> <comp>8.226138929210116e-13</comp>  </nuclide>
<nuclide>  <id>310690000.0</id> <comp>6.144217912381117e-13</comp>  </nuclide>
<nuclide>  <id>310710000.0</id> <comp>1.0846598322241

In [18]:
#printing in format for Cyclus recipe
for nuclide in spent_mox.nuclides:
    if nuclide.percent <= 1e-15:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent/100}</comp>  </nuclide>")

<nuclide>  <id>10020000</id> <comp>1.3341724827382235e-06</comp>  </nuclide>
<nuclide>  <id>20040000</id> <comp>0.00044081147519830935</comp>  </nuclide>
<nuclide>  <id>60120000</id> <comp>1.3452612273107447e-11</comp>  </nuclide>
<nuclide>  <id>60130000</id> <comp>0.0004405439407275926</comp>  </nuclide>
<nuclide>  <id>70150000</id> <comp>2.4753774946188806e-10</comp>  </nuclide>
<nuclide>  <id>80170000</id> <comp>7.317886757780683e-06</comp>  </nuclide>
<nuclide>  <id>290650000</id> <comp>3.3594678941048824e-16</comp>  </nuclide>
<nuclide>  <id>300660000</id> <comp>5.803210669938235e-14</comp>  </nuclide>
<nuclide>  <id>300670000</id> <comp>3.356982855397215e-13</comp>  </nuclide>
<nuclide>  <id>300680000</id> <comp>4.477218494879572e-13</comp>  </nuclide>
<nuclide>  <id>300700000</id> <comp>8.010740325622486e-13</comp>  </nuclide>
<nuclide>  <id>310690000</id> <comp>5.983321383495849e-13</comp>  </nuclide>
<nuclide>  <id>310710000</id> <comp>1.0562623119481054e-12</comp>  </nuclide>